<a href="https://colab.research.google.com/github/mojtabaSefidi/Fall-Detection-System/blob/master/Fall_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries and frameworks

In [ ]:
from sklearn import preprocessing
import tensorflow as tf
from math import sqrt
import pandas as pd
import numpy as np
import glob
import os
window_size = 200


Get the sisfall and sisfall_enhanced dataset

In [ ]:
!gdown --id 1kyTRhIFhqwRkf9gERof1Xm5FVQ-klLVA
!gdown --id 1gvOuxPc8dNgTnxuvPcVuCKifOf98-TV0
!unzip SisFall_dataset.zip
!unzip SisFall_enhanced.zip

# Implementation

Get all addresses

In [ ]:
def get_file_name(path):
  allfiles = []
  allFolders = glob.glob(path + "*")
  for files in allFolders:
      allfiles.append(glob.glob(files+"/*.txt"))
  if 'desktop.ini' in allfiles:
        allfiles.remove('desktop.ini')
  return np.hstack(allfiles)

In [ ]:
get_file_name("SisFall_dataset/")

array(['SisFall_dataset/SE01/D11_SE01_R02.txt',
       'SisFall_dataset/SE01/D04_SE01_R01.txt',
       'SisFall_dataset/SE01/D12_SE01_R05.txt', ...,
       'SisFall_dataset/SE09/D07_SE09_R04.txt',
       'SisFall_dataset/SE09/D17_SE09_R01.txt',
       'SisFall_dataset/SE09/D08_SE09_R05.txt'], dtype='<U37')

read dataset from address path

In [ ]:
def read_data(data_path):
    data = pd.read_csv(data_path, header=None)
    data.columns = ['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z', 'MMA8451Q_x',
                    'MMA8451Q_y', 'MMA8451Q_z']
    data['MMA8451Q_z'] = data['MMA8451Q_z'].map(lambda x: str(x)[:-1])
    for name in data.columns :
      data[name] = data[name].astype(float)
    return data

Feature Extraction

In [ ]:
def add_features(dataset,data_path):
    new_dataset = pd.DataFrame()
    new_dataset['acc_1'] = dataset.apply(
        lambda row: sqrt((row.ADXL345_x ** 2 + row.ADXL345_y ** 2 + row.ADXL345_z ** 2)), axis=1)
    new_dataset['acc_2'] = dataset.apply(
        lambda row: sqrt((row.MMA8451Q_x ** 2 + row.MMA8451Q_y ** 2 + row.MMA8451Q_z ** 2)), axis=1)
    new_dataset['geo'] = dataset.apply(
        lambda row: sqrt((row.ITG3200_x ** 2 + row.ITG3200_y ** 2 + row.ITG3200_z ** 2)), axis=1)
    new_dataset['label'] = get_label(data_path)
    return np.round(new_dataset.to_numpy(),2)


Get the Label

In [ ]:
def get_label(data_path):
    label = data_path[21]
    if label =='D':
      return int(0)
    elif label =='F':  
      label_path = data_path.replace('dataset','enhanced')
      labels = pd.read_csv(label_path,header=None)
      labels[labels == 2] = 1
      return labels


Split Dataset to train and test

In [ ]:
def split_address(dataset_address):
  np.random.shuffle(dataset_address)
  train, test = np.split(dataset_address, [int(len(dataset_address)*0.7)])
  return train, test

Extract features from All addresses

In [ ]:
def datasets_to_nparray(datasets_address_array):
  result = np.empty((0, 4), int)
  for address in datasets_address_array:
    result = np.concatenate(
        (result,add_features(read_data(address),address)),axis=0)
  return result

Windowing of the dataset

In [ ]:
def windowing(dataset,window_size):
  window = window_size * (dataset.shape[1]-1)
  cut = dataset.shape[0] % window_size
  feature = dataset[:-cut,0:-1]
  label = dataset[:-cut,-1]
  feature = feature.ravel().reshape(feature.size//window,window)
  label = label.reshape(label.size//window_size,window_size)
  label = label.sum(axis=1)
  label[label > 0] = 1
  return feature,label

Save the Train and Test Dataset as Tensor

In [ ]:
def dataset_to_tensor(test,train,window_size):
  test_feature , test_label = windowing(datasets_to_nparray(test),window_size)
  np.savez('Sisfall_data_test', inputs=test_feature, targets=test_label)
  train_feature , train_label = windowing(datasets_to_nparray(train),window_size)
  np.savez('Sisfall_data_train', inputs=train_feature, targets=train_label)


# Main

Data Pre-processing

In [ ]:
train, test = split_address(get_file_name("SisFall_dataset/"))

In [ ]:
dataset_to_tensor(test,train,window_size)

In [ ]:
npz = np.load("Sisfall_data_train.npz")
train_inputs = preprocessing.scale(npz["inputs"].astype(np.float))
train_targets = npz["targets"].astype(np.int)


npz = np.load("Sisfall_data_test.npz")
test_inputs = preprocessing.scale(npz["inputs"].astype(np.float))
test_targets = npz["targets"].astype(np.int)

In [ ]:
print(train_inputs.shape)
print(train_targets.shape)
print(test_inputs.shape)
print(test_targets.shape)

(55365, 600)
(55365,)
(23929, 600)
(23929,)


Neural Network Training

In [ ]:
input_size = 600
output_size = 1
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(input_size,activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 50

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          shuffle = True,
          verbose = 1
          )

Epoch 1/50
554/554 [==============================] - 6s 9ms/step - loss: 0.1010 - accuracy: 0.9687
Epoch 2/50
554/554 [==============================] - 8s 14ms/step - loss: 0.0686 - accuracy: 0.9765
Epoch 3/50
554/554 [==============================] - 7s 12ms/step - loss: 0.0624 - accuracy: 0.9785
Epoch 4/50
554/554 [==============================] - 7s 12ms/step - loss: 0.0589 - accuracy: 0.9790
Epoch 5/50
554/554 [==============================] - 8s 14ms/step - loss: 0.0548 - accuracy: 0.9809
Epoch 6/50
554/554 [==============================] - 6s 11ms/step - loss: 0.0525 - accuracy: 0.9817
Epoch 7/50
554/554 [==============================] - 7s 13ms/step - loss: 0.0502 - accuracy: 0.9822
Epoch 8/50
554/554 [==============================] - 8s 15ms/step - loss: 0.0472 - accuracy: 0.9836
Epoch 9/50
554/554 [==============================] - 7s 12ms/step - loss: 0.0473 - accuracy: 0.9835
Epoch 10/50
554/554 [==============================] - 6s 10ms/step - loss: 0.0441 - accurac

Neural Network Evaluation

In [ ]:
loss, accuracy = model.evaluate(test_inputs, test_targets, verbose=1)

748/748 [==============================] - 4s 5ms/step - loss: 0.1198 - accuracy: 0.9791


Confusion Matrix

In [ ]:
predictions = model.predict(test_inputs)
conf_matrix = tf.math.confusion_matrix(labels=test_targets,
                                       predictions=predictions).numpy()
print("------ Confusion Matrix Created ------")
print()
print(conf_matrix)
print()

precision_1 = (conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[0][1])).round(2)
recall_1 = (conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])).round(2)
f1Score_1 = ((2 * precision_1 * recall_1)/(precision_1 + recall_1)).round(2)
statics = pd.DataFrame([[precision_1,recall_1,f1Score_1]],columns=["Precision","Recall","F1score"])
statics

------ Confusion Matrix Created ------

[[22855     2]
 [  703   369]]



,Precision,Recall,F1score
0,0.99,0.34,0.51


#### SVM and Logistic Regression

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
x = train_inputs
y = train_targets
x_test = test_inputs
y_test = test_targets

Logistic Regression

In [ ]:
reg = LogisticRegression()
reg.fit(x,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

Logistic Regression Evaluation

In [ ]:
print("score on test: " + str(reg.score(x_test, y_test)))
print("score on train: "+ str(reg.score(x, y)))
print()
y_pred = reg.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.9545739479292908
score on train: 0.9593064210241127

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     22857
           1       0.48      0.17      0.25      1072

    accuracy                           0.95     23929
   macro avg       0.72      0.58      0.61     23929
weighted avg       0.94      0.95      0.94     23929

[[22658   199]
 [  888   184]]


Svm Model


In [ ]:
svm=LinearSVC(C = 0.0001)
svm.fit(x, y)


LinearSVC(C=0.0001)

SVM Model Evaluation

In [ ]:
print("score on test: "  + str(svm.score(x_test, y_test)))
print("score on train: " + str(svm.score(x, y)))
print()
y_pred = svm.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.9542396255589453
score on train: 0.9542851982299286

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     22857
           1       0.30      0.02      0.03      1072

    accuracy                           0.95     23929
   macro avg       0.63      0.51      0.50     23929
weighted avg       0.93      0.95      0.93     23929

[[22817    40]
 [ 1055    17]]


KNN implementation

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(x, y)

KNeighborsClassifier(n_neighbors=4)

KNN evaluation

In [ ]:
y_pred = knn.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     22857
           1       0.94      0.54      0.69      1072

    accuracy                           0.98     23929
   macro avg       0.96      0.77      0.84     23929
weighted avg       0.98      0.98      0.97     23929

[[22817    40]
 [  492   580]]


Balancing the Dataset

In [ ]:
new_train_dataset = np.concatenate((train_inputs,train_targets.reshape((train_targets.shape[0],1))),axis=1)
df = pd.DataFrame(new_train_dataset, columns = None)
df.columns = [*df.columns[:-1], 'label']
reps = [305 if val == 1 else 1 for val in df.label]
df = df.loc[np.repeat(df.index.values, reps)].reset_index(drop=True)
Balanced_train = df.to_numpy()
print('Shape before Balancing :', new_train_dataset.shape)
print('Shape After Balancing:', Balanced_train.shape)

Shape before Balancing : (55365, 601)
Shape After Balancing: (836949, 601)


Train the model with Balanced dataset

In [ ]:
input_size = 600
output_size = 1
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(input_size,activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 50

model.fit(Balanced_train[:,:-1],
          Balanced_train[:,-1],
          batch_size=batch_size,
          epochs=max_epochs,
          shuffle = True,
          verbose = 1
          )

Epoch 1/50
8370/8370 [==============================] - 59s 7ms/step - loss: 0.0494 - accuracy: 0.9845
Epoch 2/50
8370/8370 [==============================] - 57s 7ms/step - loss: 0.0275 - accuracy: 0.9924
Epoch 3/50
8370/8370 [==============================] - 57s 7ms/step - loss: 0.0204 - accuracy: 0.9945
Epoch 4/50
8370/8370 [==============================] - 58s 7ms/step - loss: 0.0164 - accuracy: 0.9958
Epoch 5/50
8370/8370 [==============================] - 58s 7ms/step - loss: 0.0137 - accuracy: 0.9965
Epoch 6/50
8370/8370 [==============================] - 58s 7ms/step - loss: 0.0111 - accuracy: 0.9972
Epoch 7/50
8370/8370 [==============================] - 58s 7ms/step - loss: 0.0098 - accuracy: 0.9975
Epoch 8/50
8370/8370 [==============================] - 58s 7ms/step - loss: 0.0087 - accuracy: 0.9978
Epoch 9/50
8370/8370 [==============================] - 59s 7ms/step - loss: 0.0074 - accuracy: 0.9982
Epoch 10/50
8370/8370 [==============================] - 59s 7ms/step - l

Testing the model

In [ ]:
loss, accuracy = model.evaluate(test_inputs, test_targets, verbose=1)


748/748 [==============================] - 2s 2ms/step - loss: 0.4430 - accuracy: 0.9763


Evaluation of the model

In [ ]:
predictions = model.predict(test_inputs)
conf_matrix = tf.math.confusion_matrix(labels=test_targets,
                                       predictions=predictions).numpy()
print("------ Confusion Matrix Created ------")
print()
print(conf_matrix)
print()

precision_2 = (conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[0][1])).round(2)
recall_2 = (conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])).round(2)
f1Score_2 = ((2 * precision_2 * recall_2)/(precision_2 + recall_2)).round(2)
statics = pd.DataFrame([[precision_2,recall_2,f1Score_2]],columns=["Precision","Recall","F1score"])
statics

------ Confusion Matrix Created ------

[[22774    83]
 [  450   622]]



,Precision,Recall,F1score
0,0.88,0.58,0.7


Balancing the train dataset for Svm and Logistic Regression Model

In [ ]:
new_train_dataset = np.concatenate((train_inputs,train_targets.reshape((train_targets.shape[0],1))),axis=1)
df = pd.DataFrame(new_train_dataset, columns = None)
df.columns = [*df.columns[:-1], 'label']
reps = [128 if val == 1 else 1 for val in df.label]
df = df.loc[np.repeat(df.index.values, reps)].reset_index(drop=True)
Balanced_train = df.to_numpy()
print('Shape before Balancing :', new_train_dataset.shape)
print('Shape After Balancing:', Balanced_train.shape)

Shape before Balancing : (55365, 601)
Shape After Balancing: (381882, 601)


Logistic Regression After Balancing

In [ ]:
reg = LogisticRegression()
reg.fit(Balanced_train[:,:-1],Balanced_train[:,-1])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

Logistic Regression Evaluation

In [ ]:
print("score on test: "  + str(reg.score(x_test, y_test)))
print("score on train: " + str(reg.score(Balanced_train[:,:-1],Balanced_train[:,-1])))
y_pred = reg.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.4821346483346567
score on train: 0.9038158384003435
              precision    recall  f1-score   support

           0       0.99      0.46      0.63     22857
           1       0.07      0.92      0.14      1072

    accuracy                           0.48     23929
   macro avg       0.53      0.69      0.38     23929
weighted avg       0.95      0.48      0.61     23929

[[10555 12302]
 [   90   982]]


SVM After Balancing

In [ ]:
svm=LinearSVC(C=0.0001)
svm.fit(Balanced_train[:,:-1],Balanced_train[:,-1])


LinearSVC(C=0.0001)

SVM Evaluation

In [ ]:
print("score on test: " + str(svm.score(x_test, y_test)))
print("score on train: "+ str(svm.score(Balanced_train[:,:-1], Balanced_train[:,-1])))
y_pred = svm.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.28045467842367
score on train: 0.8886802729638998
              precision    recall  f1-score   support

           0       0.99      0.25      0.40     22857
           1       0.06      0.96      0.11      1072

    accuracy                           0.28     23929
   macro avg       0.52      0.60      0.25     23929
weighted avg       0.95      0.28      0.38     23929

[[ 5681 17176]
 [   42  1030]]


Balancing the dataset for KNN model

In [ ]:
new_train_dataset = np.concatenate((train_inputs,train_targets.reshape((train_targets.shape[0],1))),axis=1)
df = pd.DataFrame(new_train_dataset, columns = None)
df.columns = [*df.columns[:-1], 'label']
reps = [64 if val == 1 else 1 for val in df.label]
df = df.loc[np.repeat(df.index.values, reps)].reset_index(drop=True)
Balanced_train = df.to_numpy()
print('Shape before Balancing :', new_train_dataset.shape)
print('Shape After Balancing:', Balanced_train.shape)

Shape before Balancing : (55365, 601)
Shape After Balancing: (217338, 601)


Balance KNN implementation

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(Balanced_train[:,:-1],Balanced_train[:,-1])

KNeighborsClassifier(n_neighbors=4)

Balanced KNN evaluation

In [ ]:
y_pred = knn.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     22857
           1       0.72      0.74      0.73      1072

    accuracy                           0.98     23929
   macro avg       0.85      0.86      0.86     23929
weighted avg       0.98      0.98      0.98     23929

[[22549   308]
 [  283   789]]


Prediction based on Ensemble Concept

In [ ]:
def ensmble_concept(Knn_prediction,model_prediction):
  return np.logical_or(Knn_prediction , model_prediction.T.ravel().round())

result = ensmble_concept(y_pred,predictions)
print(classification_report(y_test, result))
print(confusion_matrix(y_test, result))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     22857
           1       0.65      0.81      0.72      1072

    accuracy                           0.97     23929
   macro avg       0.82      0.90      0.85     23929
weighted avg       0.98      0.97      0.97     23929

[[22379   478]
 [  201   871]]


Models' Summary

In [ ]:
conclusion = pd.DataFrame([['Neural Network',precision_1,recall_1,f1Score_1],
              ['Logistic Regression',0.53,0.17,0.26],
              ['SVM',0.38,0.01,0.03],
              ['KNN',0.94,0.56,0.70],
              ['Neural Network after Balancing',precision_2,recall_2,f1Score_2],
              ['Logistic Regression after Balancing',0.08,0.93,0.15],
              ['SVM after Balancing',0.06,0.97,0.11],
              ['KNN after Balancing',0.73,0.73,0.73],
              ['Ensemble concept after Balancing',0.65,0.81,0.72]],
              columns=["Algorithm","Precision","Recall","F1score"])
conclusion = conclusion.set_index('Algorithm')
conclusion.style.background_gradient(cmap="YlOrRd")


,Precision,Recall,F1score
Algorithm,,,
Neural Network,0.990000,0.340000,0.510000
Logistic Regression,0.530000,0.170000,0.260000
SVM,0.380000,0.010000,0.030000
KNN,0.940000,0.560000,0.700000
Neural Network after Balancing,0.880000,0.580000,0.700000
Logistic Regression after Balancing,0.080000,0.930000,0.150000
SVM after Balancing,0.060000,0.970000,0.110000
KNN after Balancing,0.730000,0.730000,0.730000
Ensemble concept after Balancing,0.650000,0.810000,0.720000


As you see After Blancing the dataset Ensemble concept and KNN model do the best !